In [1]:
import ollama
import gradio as gr
from bs4 import BeautifulSoup
import requests

In [19]:


class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
obj=Website("https://vignan.ac.in/newvignan/")

In [20]:


system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits."

In [15]:
def msg_llama(prompt):
    result=ollama.chat(model="llama3.2",
                       messages=[{"role":"system","content":system_message},
                                 {"role":"user","content":prompt}],stream=True)
    res=""
    for i in result:
        res+=i["message"]["content"] or ""
        yield res

def msg_deepseek(prompt):
    result=ollama.chat(model="deepseek-r1",messages=[{"role":"system","content":system_message},
                                 {"role":"user","content":prompt}],stream=True)
    res=""
    for i in result:
        res+=i["message"]["content"] or ""
        yield res

In [16]:
def stream_brochure(company_name,url,model):
    prompt=f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt+="Use proper headings, bullet points, and bold text where appropriate."
    prompt += Website(url).get_contents()
    if model=="llama3.2":
        response=msg_llama(prompt)
    else:
        response=msg_deepseek(prompt)
    yield from response
        

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["llama3.2", "deepseek-r1"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never",
    title="AI-Powered Brochure Generator",
    description="Generate a personalized marketing brochure using cutting-edge language models like LLaMA 3.2 and DeepSeek-R1. Just enter your company name and website!"
)

view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://bcc7a2347ae2603426.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
